## Make your own heatmap based on Strava activities
This notebook shows you how to create your own heatmap based on your Strava activities.

You need to create a Strava API application in order to use their API. Follow the instructions on this page to create your app: <https://medium.com/@annthurium/getting-started-with-the-strava-api-a-tutorial-f3909496cd2d>

After setting up the app, note down the following information (you will need it to run this notebook):
- Client id
- Client secret

Enter the values in cell 4, and run the notebook (depending on how many activities you have in your Strava account, this might take several minutes).

**Note:** Strava imposes some request limits (30'000/day, and 600/every 15min). 

In [ ]:
!pip install stravaio folium

In [ ]:
import os
import folium
from stravaio import StravaIO
from stravaio import strava_oauth2

In [ ]:
os.environ['STRAVA_CLIENT_ID'] = "ENTER-YOUR-CLIENT-ID"
os.environ['STRAVA_CLIENT_SECRET'] = "ENTER-YOUR-CLIENT-SECRET"

In [ ]:
# This will open a browser, asking for permission to access your Strava account, click authorize to continue
auth = strava_oauth2(client_id=os.environ["STRAVA_CLIENT_ID"], client_secret=os.environ["STRAVA_CLIENT_SECRET"])

In [ ]:
client = StravaIO(access_token=auth["access_token"])
athlete = client.get_logged_in_athlete()
activities = client.get_logged_in_athlete_activities(after=20170101)

In [ ]:
m = folium.Map(
    name="Strava Heatmap",
    tiles="cartodbdark_matter",
    location=[59.925,10.728123],
    zoom_start=11.5,
    control_scale=True
)

folium.TileLayer("cartodbdark_matter").add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
for a in activities:
    if a.type == "Workout":
        continue
    streams = client.get_activity_streams(a.id, athlete.id)
    try:
        points = list(zip(streams.lat, streams.lng))
        if a.type == "Run":
            folium.PolyLine(
                locations=points, color="#ff9933", opacity=1 / 2, weight=1
            ).add_to(m)
        elif a.type == "Ride":
            folium.PolyLine(
                locations=points, color="#0066ff", opacity=1 / 2, weight=1
            ).add_to(m)
        elif a.type == "NordicSki":
            folium.PolyLine(
                locations=points, color="#00ffff", opacity=1 / 2, weight=1
            ).add_to(m)
        elif a.type == "AlpineSki":
            folium.PolyLine(
                locations=points, color="#00ccff", opacity=1 / 2, weight=1
            ).add_to(m)
        elif a.type == "Canoeing":
            folium.PolyLine(
                locations=points, color="#00ff55", opacity=1 / 2, weight=1
            ).add_to(m)
        elif a.type == "IceSkate":
            folium.PolyLine(
                locations=points, color="#f6ff00", opacity=1 / 2, weight=1
            ).add_to(m)
        else:
            folium.PolyLine(
                locations=points, color="#cc00ff", opacity=1 / 2, weight=1
            ).add_to(m)
        logging.critical("Mapped activity with id: {}".format(a.id))
    except:
        logging.error("Could not map activity with id: {}".format(a.id))

In [ ]:
m